# Putting it all together - Agent Execution

Building on our previous steps where we set up the Llama Stack environment, registered MCP tools, and established our Vector Database for RAG, this section brings all these components together.

We will now focus on **Agent Execution**, demonstrating how a sophisticated AI agent, powered by the ReAct framework, can leverage these resources to process complex queries. This is where you will see the combined power of the Llama Stack ecosystem in action.

In this section, you will:

* **Configure a ReAct agent** designed to work with multiple toolgroups and a tailored prompt.
* **Observe the agent's decision-making process** as it selects and utilizes different tools based on the user's input.
* **Examine the agent's "Chain of Thought"** to understand how it approaches and breaks down problems.
* **See how RAG is integrated** to provide the agent with access to domain-specific knowledge for informed responses.
* **Understand the synergy** of combining Llama Stack's core components to build intelligent, multi-capability applications.

Let's connect the pieces and run our advanced Llama Stack agent!

***

In [ ]:
!pip install -U llama-stack-client==0.2.5 dotenv > /dev/null 2>&1 && echo "pip Python Prerequisites installed succesfuly"

import os
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# These libraries are just here to print the results from the agent in a more human-readable way 
from src.utils import step_printer
#from src.client_tools import get_location
from termcolor import cprint
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
stream=False ## Defaulting to False, you can change this throughout the section to "True" if you wanted to see the output in another format (Using EventLogger)

# for our lab, we will just define our variables manualy here, in a regular application, this would be ready directly from the local .env file and we would comment these lines out
os.environ['LLAMA_STACK_SERVER'] = 'http://localhost:8321'
from llama_stack_client import LlamaStackClient
LLAMA_STACK_SERVER=os.getenv("LLAMA_STACK_SERVER")

client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
)

# List available models and select from allowed models list
allowed_models_list=["granite3.2:8b"]
selected_model = None

models = client.models.list()
print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")
    # Check if the model identifier contains any of the allowed substrings
    if any(substring in m.identifier for substring in allowed_models_list):
        # Only set selected_model if it hasn't been set yet
        if selected_model is None:
            selected_model = m.identifier
           
# If no allowed model was found, you might want to handle that case
if selected_model is None:
    print("No allowed model found in the list.")
print(f"Selected model (from allowed list): {selected_model}")
model = selected_model

vector_db_id = "Our_Parks_DB"
query_config = {
    "query_generator_config": {
        "type": "default",
        "separator": " "
    },
    "max_tokens_in_context": 300,
    "max_chunks": 2
}


### Customizing the Agent's Instructions (The Prompt)

The prompt serves as the core instructions for our AI agent, guiding its behavior and strategy. While Llama Stack provides default prompts, we have the flexibility to define our own to precisely control how the agent operates and interacts with its tools.

In this step, we will **define a custom prompt string**. This string includes placeholders (`<<tool_names>>`, `<<tool_descriptions>>`) where we will dynamically insert the details of the tools available to the agent. This explicit approach ensures the agent is fully aware of its capabilities and the specific rules we define for tool usage.

This exercise highlights how you can tailor the agent's foundational instructions to meet specific application requirements.

> **Note:**
> To see how the default ReAct agent prompt is structured, you can explore the source code [here](https://github.com/meta-llama/llama-stack-client-python/blob/main/src/llama_stack_client/lib/agents/react/prompts.py).


In [ ]:
custom_react_prompt = """

You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.

To do so, you have been given access to the following tools: <<tool_names>>

🚨 TOOL USAGE RULES — FOLLOW STRICTLY 🚨

1. ❌ Do NOT guess or invent tool names.
2. ✅ You may only call tools that are explicitly listed in <<tool_names>>.
3. 🛑 Never use tools like `Maps`, `search`, `maps_geocode`, or `knowledge_search` unless they are explicitly in <<tool_names>>.
4. If you cannot solve a task using the tools available, explain that limitation in your final answer instead of calling an invalid tool.
5. 🔎 ALWAYS ALWAYS ALWAYS USE `get_park_location` before `get_alerts`
6. ❓ If a tool needs location data (like `get_alerts`, `maps_search_places`), you must first call `get_park_location` and extract the relevant state, city, or coordinates from its result before proceeding. You must not hardcode, assume, or guess this data.
7. 🧠 Internally keep track of whether you have already retrieved park location. Do not call `get_alerts` until this location has been confirmed via `get_park_location`.

🛦 TOOL PARAMETER FORMAT (MANDATORY):
Each tool call must use the following format for `tool_params`:
```json
"tool_params": [
  {"name": "parameter_name", "value": "actual value"}
]
```

---

🧐 RESPONSE FORMAT (ALWAYS JSON):

{
    "thought": $THOUGHT_PROCESS,
    "action": {
        "tool_name": $TOOL_NAME,
        "tool_params": $TOOL_PARAMS
    },
    "answer": $ANSWER
}

Only one tool may be called at a time. Use multiple steps when needed.

Use `"action": null` and set `answer` when you’re ready to respond to the user.

---

🗓️ EXAMPLE:

Task: “Are there any supermarkets near Crimson Basin?”

Step 1:
{
    "thought": "I need to get the location of Crimson Basin park using get_park_location.",
    "action": {
        "tool_name": "get_park_location",
        "tool_params": [
            {"name": "park_name", "value": "Crimson Basin"}
        ]
    },
    "answer": null
}

Observation: {"result": "Crimson Basin is located in Nevada, USA."}

Step 2:
{
    "thought": "Now I will search for supermarkets near that location using maps_search_places.",
    "action": {
        "tool_name": "maps_search_places",
        "tool_params": [
            {"name": "query", "value": "supermarkets near Nevada, USA"}
        ]
    },
    "answer": null
}

Observation: { ... list of stores ... }

Final step:
{
    "thought": "I now have the info the user asked for.",
    "action": null,
    "answer": "There are several supermarkets near Crimson Basin, including WinCo Foods, Albertsons, and Smith’s."
}

---

❗ IF A TOOL IS MISSING:

If no tool is available for the task:
{
    "thought": "I need to get nearby supermarkets, but I do not have access to a maps tool.",
    "action": null,
    "answer": "I’m unable to find supermarkets near Crimson Basin because I don’t have access to a maps tool."
}

---

You only have access to the following tools: <<tool_descriptions>>

SUMMARY:
- Do not guess tools. Use only the ones listed.
- Follow the exact `tool_params` format.
- Use get_park_location to resolve any location before calling tools that require geographic input.
- Do not hardcode, guess, or assume state names or coordinates.
- Always use get_park_location before get_alerts.
- Keep internal memory of what locations have already been retrieved.
- Return final answers with `"action": null`.

Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000.

"""


### Identifying Available Tools for the Agent

Now that we have our custom prompt defined, we need to determine which tools the agent will have at its disposal. Llama Stack makes it straightforward to manage tools registered via MCPs or other sources.

In this step, we will **retrieve the list of available tools** from the Llama Stack server. When MCP servers are registered, Llama Stack interacts with them to automatically discover and catalog the tools they expose, including their detailed descriptions and how to use them (parameters, etc.).

We will then filter this complete list to select only the specific toolgroups required for our agent in this lab.

In [ ]:
registered_tools = client.tools.list()


allowed_toolgroups = [
    'mcp::mcp-weather',
    'mcp::mcp-googlemaps',
    'mcp::mcp-parks-info',
]

# Initialize an empty list to store the dictionaries for each tool
allowed_tools_array = []

for tool in registered_tools:
    if tool.toolgroup_id in allowed_toolgroups:
        #print(tool)
        allowed_tools_array.append(tool)

# If you want to see what this looks like, uncomment these lines
#print("List of allowed tools (each as a dictionary):")
#print(allowed_tools_array)


### Dynamically Injecting Tool Information

Normally, Llama Stack handles the integration of tool information into the agent's context automatically. However, since we are using a completely custom prompt in this lab, we need to manually perform this step.

This process demonstrates the flexibility Llama Stack offers. By manually injecting the tool names and descriptions (which we retrieved in the previous step) into the `<<tool_names>>` and `<<tool_descriptions>>` placeholders in our custom prompt string, we are explicitly providing the agent with the definition of its capabilities.

This ensures that the agent, guided by our custom instructions and now equipped with knowledge of its available tools, is ready to process user requests intelligently.

In [ ]:
import re

def insert_tools_to_prompt(my_instructions: str, allowed_tools_array: list) -> str:
    """
    Formats the source template string by inserting tool names and descriptions
    from a list of Tool objects.

    Args:
        my_instructions: A multi-string variable containing the source template.
                         Expected to have <<tool_names>> and <<tool_descriptions>> placeholders.
        allowed_tools_array: A list of Tool objects obtained from the client library.

    Returns:
        A multi-string variable with placeholders replaced by formatted tool information.
    """
    tool_names = []
    tool_descriptions = []

    for tool in allowed_tools_array:
        tool_names.append(tool.identifier)
        formatted_parameters = []
        for param in tool.parameters:
            # Escape single quotes in the parameter description for the string literal
            param_description_escaped = param.description.replace("'", "\\'")
            formatted_parameters.append(
                f"Parameter(description='{param_description_escaped}', name='{param.name}', parameter_type='{param.parameter_type}', required={param.required}, default={param.default})"
            )
        cleaned_description = tool.description.replace('\\n', '\n').replace("    ", "").replace("'", "\\'")

        tool_descriptions.append(
            f"- {tool.identifier}: {{'name': '{tool.identifier}', 'description': '{cleaned_description}', 'parameters': [{', '.join(formatted_parameters)}]}}"
        )

    tool_names_string = ", ".join(tool_names)

    tool_descriptions_string = "\n".join(tool_descriptions)

    output_template = my_instructions.replace("<<tool_names>>", tool_names_string).replace("<<tool_descriptions>>", tool_descriptions_string)

    return output_template


In [ ]:
custom_react_prompt_with_tools=insert_tools_to_prompt(custom_react_prompt,allowed_tools_array)
print(custom_react_prompt_with_tools)

## Unleashing the Custom ReAct Agent

Alright, the stage is set! We have our Llama Stack server, our specialized MCP tools are registered, our knowledge base is ready, and we've crafted a custom prompt loaded with tool definitions.

Now, it's time to bring our **ReAct agent** to life and see it in action! In this section, we'll instantiate the `ReActAgent` class with our configurations and run a few example queries. Pay close attention to the output – you'll be able to follow the agent's reasoning process and how it decides to use the tools we provided.

*(That's where the "ReAct" name comes from – it's a framework focused on the agent's **Reasoning** and **Acting** by calling tools!)*

> **Important Considerations:**
> 
> Keep in mind that the agent's responses can vary based on the specific question, the underlying language model, the prompt design, and even settings like temperature. This exercise is designed to illustrate the *potential* of building intelligent agents with Llama Stack. Creating a production-ready, perfectly conversational agent often involves further iteration and fine-tuning of these components. Embrace the journey and observe the process!


In [ ]:
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput

stream=False


agent = ReActAgent(
            client=client,
            model=model,
            instructions=custom_react_prompt_with_tools,
            tools=["mcp::mcp-parks-info","mcp::mcp-weather","mcp::mcp-googlemaps"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            #sampling_params=sampling_params,
        )
user_prompts = [
    #"are there any hotels on the drive from Las Vegas, Nevada to Crimson Basin park",
     "Are there any supermarkets near Crimson Basin park?", # Works
    # "Should I be worried about severe weather conditions around Azure Mongrove Wilderness park today?",
    # "Can you find a pharmacy close to Prismatic Painted Prairie park?",
    # "what are the coordinates of Prismatic Painted Prairie park?",
]


for prompt in user_prompts:
    new_uuid = uuid.uuid4()
    session_id = agent.create_session(f"React-session-{new_uuid}")
    print("\n"+"="*50)
    print(f"Processing user query: {prompt}", "blue")
    print("="*50)


    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


# Putting it all together - Agent Execution

With our environment successfully prepared in the previous section, it's time for the main event! In this part of the lab, we will take all the pieces we've assembled – our Llama Stack server, the registered MCP tools providing external capabilities, our populated Vector Database for RAG, and our understanding of ReAct agents – and put them into action.

This section is where you'll see the true power of Llama Stack agents come to life as we build and execute a sophisticated agent capable of reasoning, interacting with multiple external services, and accessing domain-specific knowledge to answer complex queries.

By working through this section, you will:

* See how to configure a ReAct agent with multiple toolgroups and a custom prompt.
* Understand how an agent selects and utilizes different tools based on the user's request.
* Observe the agent's "Chain of Thought" as it breaks down a complex problem.
* Experience how RAG is integrated into the agent's workflow to provide informed responses.
* Appreciate the flexibility and power of combining various Llama Stack components to build intelligent applications.

Let's connect everything and unleash the potential of our multi-talented Llama Stack agent!

***